# SOU Exploratory Analyis & File Creation

### Libraries

In [107]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pdfplumber
import re

# Personal Lib for this project
from tools import *

### Testing on One File
- TODO:
    Compile everything into one function

In [108]:
# Path to for rep file
pdf_path = "../data/pdf/March1.pdf"

with pdfplumber.open(pdf_path, pages = [1]) as pdf:
    # Extract text from each page
    text = ""
    for page in pdf.pages:
        text += page.extract_text();

In [109]:
matches = re.findall(r'Southern Oregon \d+..(#[\s\S]+?..TM Team)', text, re.DOTALL)
matches_date = re.findall(r'Date: \s*(........)', text)
matches

['# Player GS MIN FG 3PT FT ORB-DRB REB PF A TO BLK STL PTS\n20 Morgan Baird * 24 6-9 0-0 4-4 4-3 7 0 1 1 1 0 16\n10 Bridgette McIntyre * 32 4-7 3-4 0-0 0-5 5 1 4 3 0 0 11\n1 Meghan McIntyre * 31 4-13 0-3 0-0 4-6 10 2 3 4 0 0 8\n12 Emma Schmerbach * 24 3-12 0-2 0-0 1-9 10 3 3 1 0 2 6\n23 Clara Robbins * 17 1-3 0-0 0-0 0-3 3 0 1 0 0 2 2\n22 Sierra Scheppele 24 3-6 2-5 0-0 0-1 1 2 0 0 0 0 8\n32 Eliza DiGiulio 19 1-7 0-1 4-6 1-2 3 0 1 2 0 0 6\n13 Keeley Wright 9 2-3 0-0 0-0 2-1 3 0 1 0 0 1 4\n33 Mallory Williams 10 1-1 1-1 0-0 0-3 3 0 0 0 0 0 3\n5 Makayla Scurlock 10 1-2 0-1 0-0 2-2 4 2 1 1 1 1 2\nTM Team']

In [110]:
stats = matches[0]
stats = stats.strip().split(sep = '\n')
split_stats = list(map(str.split, stats))

opponent = get_opponent(text)
title = split_stats[0] + ['Opponent'] + ['Date']

player_stats = {key: [] for key in title}

players = split_stats[1:-1]
players = list(map(merge_and_pop, players))
players = list(map(non_start, players))
players

stat_dict = {key: [] for key in title}

for player in players:

    for stat, data in zip(title, player):
        stat_dict[stat].append(data)
    stat_dict['Opponent'].append(opponent)
    stat_dict['Date'].append(matches_date[0])

# Proof of Concept
pd.DataFrame.from_dict(stat_dict)

,#,Player,GS,MIN,FG,3PT,FT,ORB-DRB,REB,PF,A,TO,BLK,STL,PTS,Opponent,Date
0,20,Morgan Baird,*,24,6-9,0-0,4-4,4-3,7,0,1,1,1,0,16,College of Idaho,03/01/25
1,10,Bridgette McIntyre,*,32,4-7,3-4,0-0,0-5,5,1,4,3,0,0,11,College of Idaho,03/01/25
2,1,Meghan McIntyre,*,31,4-13,0-3,0-0,4-6,10,2,3,4,0,0,8,College of Idaho,03/01/25
3,12,Emma Schmerbach,*,24,3-12,0-2,0-0,1-9,10,3,3,1,0,2,6,College of Idaho,03/01/25
4,23,Clara Robbins,*,17,1-3,0-0,0-0,0-3,3,0,1,0,0,2,2,College of Idaho,03/01/25
5,22,Sierra Scheppele,-,24,3-6,2-5,0-0,0-1,1,2,0,0,0,0,8,College of Idaho,03/01/25
6,32,Eliza DiGiulio,-,19,1-7,0-1,4-6,1-2,3,0,1,2,0,0,6,College of Idaho,03/01/25
7,13,Keeley Wright,-,9,2-3,0-0,0-0,2-1,3,0,1,0,0,1,4,College of Idaho,03/01/25
8,33,Mallory Williams,-,10,1-1,1-1,0-0,0-3,3,0,0,0,0,0,3,College of Idaho,03/01/25
9,5,Makayla Scurlock,-,10,1-2,0-1,0-0,2-2,4,2,1,1,1,1,2,College of Idaho,03/01/25


In [133]:
def folder_to_csv(folder_path:str, test:bool = False):
    
    if test:
        folder = [folder_path]
    else:
        folder = os.listdir(folder_path)

    i = 0 # used to track the number of files

    for file in folder:

        with pdfplumber.open(file, pages= [1]) as pdf:
            
            # Extract text from each page
            text = ""
            for page in pdf.pages:
                text += page.extract_text();
        matches = re.findall(r'Southern Oregon \d+..(#[\s\S]+?..TM Team)', text, re.DOTALL)
        matches_date = re.findall(r'Date: \s*(........)', text)

        stats = matches[0]
        stats = stats.strip().split(sep = '\n')
        split_stats = list(map(str.split, stats))

        opponent = get_opponent(text)
        title = split_stats[0] + ['Opponent'] + ['Date']

        if i == 0:
            player_stats = {key: [] for key in title}

        players = split_stats[1:-1]
        players = list(map(merge_and_pop, players))
        players = list(map(non_start, players))
        players

        if i == 0:
            stat_dict = {key: [] for key in title}

        for player in players:

            for stat, data in zip(title, player):
                stat_dict[stat].append(data)
            stat_dict['Opponent'].append(opponent)
            stat_dict['Date'].append(matches_date[0])
        i += 1

    return pd.DataFrame.from_dict(stat_dict)


In [136]:
#folder_to_csv('../data/pdf')

folder = os.listdir('../data/pdf')
folder_path = '../data/pdf'
for file in folder:
    print(folder_path + '/' + file)


../data/pdf/Jan03.pdf
../data/pdf/March1.pdf
../data/pdf/Jan17.pdf
../data/pdf/.DS_Store
../data/pdf/March4.pdf
../data/pdf/March20.pdf
../data/pdf/March22.pdf
../data/pdf/Jan04.pdf
../data/pdf/Jan10.pdf
../data/pdf/Jan11.pdf
../data/pdf/Dec14.pdf
../data/pdf/Feb7.pdf
../data/pdf/Nov23.pdf
../data/pdf/Dec06.pdf
../data/pdf/Dec07.pdf
../data/pdf/Feb1.pdf
../data/pdf/Feb14.pdf
../data/pdf/Nov21.pdf
../data/pdf/Feb15.pdf
../data/pdf/Feb18.pdf
../data/pdf/Dec21.pdf
../data/pdf/Dec20.pdf
../data/pdf/Feb26.pdf
../data/pdf/Feb22.pdf
../data/pdf/Nov16.pdf
../data/pdf/Feb8.pdf
../data/pdf/Feb21.pdf
../data/pdf/Nov15.pdf
../data/pdf/March14.pdf
../data/pdf/March15.pdf
../data/pdf/Nov1.pdf
../data/pdf/Jan25.pdf
../data/pdf/Jan31.pdf
../data/pdf/Jan24.pdf
../data/pdf/Jan18.pdf
../data/pdf/Nov2.pdf


In [120]:
folder_to_csv('../data/pdf/Jan03.pdf', test = True)

,#,Player,GS,MIN,FG,3PT,FT,ORB-DRB,REB,PF,A,TO,BLK,STL,PTS,Opponent,Date
0,20,Morgan Baird,*,21,5-12,1-2,2-2,4-3,7,1,2,0,0,0,13,Evergreen (WA),01/03/25
1,10,Bridgette McIntyre,*,26,4-7,4-7,0-0,0-3,3,0,1,2,0,1,12,Evergreen (WA),01/03/25
2,12,Emma Schmerbach,*,25,4-11,1-1,3-3,1-5,6,0,8,3,2,0,12,Evergreen (WA),01/03/25
3,1,Meghan McIntyre,*,27,4-8,1-3,0-1,0-5,5,1,3,2,0,1,9,Evergreen (WA),01/03/25
4,23,Clara Robbins,*,13,3-4,0-0,0-0,2-7,9,1,0,0,1,0,6,Evergreen (WA),01/03/25
5,34,Shakia Teague-Perry,-,6,5-6,0-0,0-0,1-0,1,0,0,0,0,1,10,Evergreen (WA),01/03/25
6,32,Eliza DiGiulio,-,14,3-5,1-1,2-2,2-0,2,0,0,1,0,0,9,Evergreen (WA),01/03/25
7,13,Keeley Wright,-,14,2-5,1-2,0-0,0-4,4,1,2,1,0,0,5,Evergreen (WA),01/03/25
8,21,Jenika Zurita,-,11,2-4,1-3,0-0,0-2,2,0,3,1,0,1,5,Evergreen (WA),01/03/25
9,22,Sierra Scheppele,-,18,0-4,0-4,1-2,0-1,1,1,2,0,0,1,1,Evergreen (WA),01/03/25


In [128]:
files = os.listdir('../data/pdf')

os.path.join('../data/pdf', files[0])

'../data/pdf/Jan03.pdf'